In [16]:
import pandas as pd

# ROBOTA
nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"
survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\relia\AppData\Local\Temp\ipykernel_15004\4111068820.py:10: DtypeWarning:

Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\relia\AppData\Local\Temp\ipykernel_15004\4111068820.py:11: DtypeWarning:

Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.



In [17]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

df_bridges_all = df_bridges_all[df_bridges_all["n_2024"] != 0]

df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024'],
      dtype='object')

In [18]:
df_bridges_all["Trieda_PK"].value_counts()

Trieda_PK
cesta III. triedy                      3967
cesta I. triedy                        1913
cesta II. triedy                       1472
diaľnica                                771
miestna neurčená                         56
účelová cesta (UC)                       13
II. trieda - miestna zberná (MZ)          2
III. trieda - miestna obslužná (MO)       1
Name: count, dtype: int64

In [19]:
filtered_bridges_I = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta I. triedy') 
]

filtered_bridges_II = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta II. triedy') |
    (df_bridges_all["Trieda_PK"] == 'cesta III. triedy')
]

filtered_bridges_I = filtered_bridges_I[filtered_bridges_I["n_2024"] != 0]
filtered_bridges_II = filtered_bridges_II[filtered_bridges_II["n_2024"] != 0]

filtered_bridges_I.shape, filtered_bridges_II.shape

((1913, 44), (5439, 44))

In [20]:
df_bridges_all["n_2024"].value_counts()

n_2024
4    3035
5    1931
3    1381
6     895
2     525
1     368
7      60
Name: count, dtype: int64

In [24]:
import plotly.graph_objects as go

# Creating the histogram for filtered_bridges
trace_filtered_I = go.Histogram(
    x=filtered_bridges_I["n_2024"],
    name='Mosty ciest I. triedy',
    marker_color='#e7a564 ',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=1  # Set offsetgroup to 1 for the bars to be side by side
)

# Creating the histogram for df_bridges_all
trace_all = go.Histogram(
    x=filtered_bridges_II["n_2024"],
    name='Mosty ciest II. a III. triedy',
    marker_color='#46a7a2',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=2  # Set offsetgroup to 2 for the bars to be side by side
)

# Creating the figure and adding the histograms
fig = go.Figure(data=[trace_filtered_I, trace_all])

# Update the layout with Montserrat font
fig.update_layout(
    title_text='Relatívne zastúpenie STS mostov',
    xaxis_title_text='stavebno-technický stav',
    yaxis_title_text='zastúpenie (%)',
    barmode='group',  # Set barmode to "group" for bars to be side by side
    font=dict(
        family="Montserrat, sans-serif",  # Set font family to Montserrat
        size=14
    ),
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=1.10,  # Position the legend slightly above the plot
        xanchor="left",  # Anchor legend to the left
        x=.50  # Align legend to the left
    ),
    plot_bgcolor='#EEEEEE',
    margin=dict(l=0, r=20, t=50, b=50),  # Adjust margins for tight layout
    bargap=0.2
)

# Adjusting the opacity for better visualization
fig.update_traces(opacity=0.75)

fig.show()